In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
import os
import json
tqdm.pandas()


In [2]:
df = pd.read_parquet("Database/01_bbdd_think_tanks.parquet")

In [3]:
df = df[(df["FechaPublicacion"] >= "2019-01-01") & (df["FechaPublicacion"] <= "2023-12-31")]

In [4]:
df.shape

(17432, 20)

In [ ]:
def tfidf_por_grupo(df_grupo, top_n=10):
    if len(df_grupo) < 2:
        return {}

    vectorizer = TfidfVectorizer(ngram_range=(3,3))
    X = vectorizer.fit_transform(df_grupo["TextoLimpio"])
    feature_names = vectorizer.get_feature_names_out()
    tfidf_df = pd.DataFrame.sparse.from_spmatrix(X, columns=feature_names)
    tfidf_df["FechaPublicacion"] = df_grupo["FechaPublicacion"].values

    resumen = {}
    diarios = tfidf_df.groupby("FechaPublicacion").mean(numeric_only=True)

    for fecha, fila in diarios.iterrows():
        top_terms = fila.sort_values(ascending=False).head(top_n)
        resumen[fecha] = {
            "valores_tfidf": top_terms.index.tolist(),
            "puntajes_tfidf": [round(x, 4) for x in top_terms.values]
        }

    return resumen

In [ ]:
def procesar_grupo(df, group_col, group_name, top_n=10):
    df_grupo = df[df[group_col] == group_name]
    resultado = tfidf_por_grupo(df_grupo, top_n=top_n)
    resultado_con_nombre = {
        (group_name, fecha): valores
        for fecha, valores in resultado.items()
    }
    return resultado_con_nombre

In [18]:
def guardar_snapshot_json(snapshot_dict, nombre_archivo):
    with open(nombre_archivo, "w", encoding="utf-8") as f:
        json.dump(snapshot_dict, f, ensure_ascii=False, indent=2)

In [19]:
conteo_tt = df["Think Tank"].value_counts()
think_tanks_ordenados = conteo_tt.index.tolist()

In [20]:
top_n = 24
top_think_tanks = think_tanks_ordenados[:top_n]

In [ ]:
for tt in tqdm(top_think_tanks, desc="Procesando Think Tanks individualmente"):
    print(f"→ Procesando {tt}")
    snapshot = procesar_grupo(df, group_col="Think Tank", group_name=tt, top_n=10)
    nombre_archivo = f"tfidf_snapshot_tt_{tt.replace(' ', '_').lower()}.json"
    guardar_snapshot_json(snapshot, nombre_archivo)
    print(f"   Guardado en {nombre_archivo}")

Procesando Think Tanks individualmente:   0%|          | 0/24 [00:00<?, ?it/s]

→ Procesando Signos Uandes
